## RF-Grid (Without Train Test Split)

In [1]:
import pandas as pd

### Data collection

In [2]:
dataset = pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


### Data preprocessing

In [4]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

### Input Output split

In [7]:
indep = dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes' ]]
dep = dataset[['charges']]

### Standardization

In [8]:
from sklearn.preprocessing import StandardScaler
SC = StandardScaler()
indep = SC.fit_transform(indep)

In [9]:
indep

array([[-1.43876426, -0.45332   , -0.90861367, -1.0105187 ,  1.97058663],
       [-1.50996545,  0.5096211 , -0.07876719,  0.98959079, -0.5074631 ],
       [-0.79795355,  0.38330685,  1.58092576,  0.98959079, -0.5074631 ],
       ...,
       [-1.50996545,  1.0148781 , -0.90861367, -1.0105187 , -0.5074631 ],
       [-1.29636188, -0.79781341, -0.90861367, -1.0105187 , -0.5074631 ],
       [ 1.55168573, -0.26138796, -0.90861367, -1.0105187 ,  1.97058663]])

### Model Creation Through GridSearchCV

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

params_grid = {'criterion' : ['squared_error', 'friedman_mse', 'absolutr_error', 'poisson'],
               'n_estimators' : [50,100,500,1000,2000],
               'max_features' : ['sqrt', 'log2']}

grid = GridSearchCV(RandomForestRegressor(), params_grid, refit = True, verbose = 3, n_jobs= -1)
grid.fit(indep,dep)


Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
50 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
34 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_p

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolutr_error', 'poisson'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100, 500, 1000, 2000]},
             verbose=3)

In [11]:
# print the best parameter after tuning
result = grid.cv_results_

grid_predictions = grid.predict(indep)

from sklearn.metrics import r2_score
R2_Score = r2_score(dep, grid_predictions)

# print(grid.best_params_)
print("Best parameter of RF-Grid {}:",format(grid.best_params_),R2_Score)

Best parameter of RF-Grid {}: {'criterion': 'friedman_mse', 'max_features': 'sqrt', 'n_estimators': 500} 0.9770084737404563


### GridSearchCV Table 

In [12]:
Total = pd.DataFrame(result)
Total

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.276674,0.009216,0.016072,0.003881,squared_error,sqrt,50,"{'criterion': 'squared_error', 'max_features':...",0.855169,0.772248,0.857702,0.830961,0.840311,0.831278,0.031097,28
1,0.565360,0.030022,0.033014,0.008695,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.857089,0.778241,0.855618,0.827756,0.838949,0.831531,0.028784,26
2,3.055611,0.094029,0.105227,0.011065,squared_error,sqrt,500,"{'criterion': 'squared_error', 'max_features':...",0.861069,0.781032,0.861870,0.830407,0.842558,0.835387,0.029637,15
3,5.813937,0.218543,0.231549,0.045895,squared_error,sqrt,1000,"{'criterion': 'squared_error', 'max_features':...",0.860249,0.781553,0.865047,0.830077,0.843056,0.835996,0.029937,4
4,12.369771,1.077615,0.475305,0.037951,squared_error,sqrt,2000,"{'criterion': 'squared_error', 'max_features':...",0.859460,0.781101,0.863720,0.830420,0.843700,0.835680,0.029741,7
5,0.272408,0.003959,0.012134,0.001147,squared_error,log2,50,"{'criterion': 'squared_error', 'max_features':...",0.859558,0.777816,0.856003,0.826852,0.836308,0.831307,0.029374,27
6,0.550578,0.029065,0.022804,0.002907,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.856771,0.779961,0.857713,0.830075,0.842998,0.833504,0.028622,21
7,3.754988,0.314959,0.122376,0.026958,squared_error,log2,500,"{'criterion': 'squared_error', 'max_features':...",0.859276,0.782003,0.863787,0.830453,0.842158,0.835535,0.029316,11
8,6.448583,0.638828,0.206151,0.029600,squared_error,log2,1000,"{'criterion': 'squared_error', 'max_features':...",0.860525,0.783167,0.863578,0.830785,0.843031,0.836217,0.029085,2
9,13.526214,0.304878,0.753945,0.109393,squared_error,log2,2000,"{'criterion': 'squared_error', 'max_features':...",0.860381,0.782275,0.862629,0.829237,0.842580,0.835421,0.029248,13


### Get the User Inputs

In [13]:
Age_input = float(input("Age : "))
Bmi_input = float(input("BMI : "))
Children_input = int(input("Children : "))
SEX_Male_input = int(input("Sex_Male 0 or 1 : "))
Smoker_YES_input = int(input("Smoker_YES 0 or 1 : "))

Age :  32
BMI :  20
Children :  2
Sex_Male 0 or 1 :  1
Smoker_YES 0 or 1 :  1


### Predictions

In [15]:
Future_predictions = grid.predict([[Age_input,Bmi_input,Children_input,SEX_Male_input,Smoker_YES_input]])
print("Future predictions of Insurance :" , Future_predictions)

Future predictions of Insurance : [46513.52004528]
